In [1]:
import os
import glob

import numpy as np
import xarray as xr
import pandas as pd

## Read statistics
from sclouds.plot.helper_bar_plot import autolabel, read_global_statistics_to_dataframe
from sclouds.helpers import VARIABLES, FILTERS, LONGNAME, path_stats_results, UNITS, path_store_plots

from sclouds.plot.helpers import import_matplotlib, TEXT_WIDTH_IN, TEXT_HEIGHT_IN
mat = import_matplotlib()
import matplotlib.pyplot as plt

STATISTICS = ['mean', 'min', 'max', 'std', 'median', 'mad']  # TODO add MAD ikveld 
FILTERS    = ['land', 'sea', 'all', 'coast']

Warning.. Using pgf backend, no GUI available. use plt.savefig() for inpection
Warning.. Using pgf backend, no GUI available. use plt.savefig() for inpection


In [ ]:
files = glob.glob(os.path.join(path_stats_results, '*global*.nc'))

In [ ]:
len(files) # skal være 25 for jeg har 5 variabler of 5 filter

In [ ]:
files

In [39]:
files

['/home/hanna/lagrings/results/stats/stats_pixel_r_coast.nc',
 '/home/hanna/lagrings/results/stats/stats_pixel_sp_all.nc',
 '/home/hanna/lagrings/results/stats/stats_pixel_tcc_all.nc',
 '/home/hanna/lagrings/results/stats/stats_pixel_t2m_all.nc',
 '/home/hanna/lagrings/results/stats/stats_pixel_r_all.nc',
 '/home/hanna/lagrings/results/stats/stats_pixel_q_all.nc',
 '/home/hanna/lagrings/results/stats/stats_pixel_r_sea.nc']

In [2]:
def retrive_stat_from_stores_statistics(statistic = 'min'):
    """Regrouped to easy acess the statistics"""
    dd = read_global_statistics_to_dataframe()
    df_stat = dd.unstack().swaplevel().unstack().swaplevel().transpose()
    return df_stat[statistic]

def retrive_filter_from_stores_statistics(filter_key = 'all'):
    """Regrouped to easy acess the statistics"""
    dd = read_global_statistics_to_dataframe()
    return dd[filter_key]

def retrive_variable_from_stores_statistics(variable = 'tcc'):
    """Regrouped to easy acess the statistics"""
    dd = read_global_statistics_to_dataframe()
    df_stat = dd.unstack().swaplevel().unstack()#.swaplevel().transpose()
    return df_stat[variable].unstack()

# Examples :

In [3]:
retrive_stat_from_stores_statistics(statistic = 'mad')

,all,artefact,coast,land,sea
q,-0.0004080399,-0.000491415329192342,-0.0004857055188470221,-0.00043330837335283506,-0.00047203663159726093
r,4.323242,2.711659273099272,2.2590836622601813,3.7136264460525155,1.8510166247728534
sp,1984.0469,332.15902424913656,383.48233831299876,992.5191701118165,35.76767987315543
t2m,0.035980225,-0.4691067408252252,-0.3820681981734424,-0.41744391266331604,-0.4709612391728797
tcc,0.24276267175225108,0.24790403517287174,0.22551782613782256,0.24129087650976022,0.24027279844578175


In [4]:
retrive_variable_from_stores_statistics(variable = 'tcc')

,mean,std,min,max,median,mad
all,0.7180500543958317,0.3582878712212896,9.713876067595185e-08,1.0,0.9608127261480828,0.24276267175225108
artefact,0.7489750682710737,0.34689479520988287,6.17552416870995e-07,1.0,0.9968791034439455,0.24790403517287174
coast,0.6860136532201572,0.3707949986218723,6.17552416870995e-07,1.0,0.9115314793579797,0.22551782613782256
land,0.7038086370226194,0.3655783380530621,2.7776128581188306e-07,1.0,0.9450995135323796,0.24129087650976022
sea,0.7323591822842174,0.3502274286143192,9.713876067595185e-08,1.0,0.9726319807299991,0.24027279844578175


In [5]:
retrive_filter_from_stores_statistics(filter_key = 'all')

,r,q,t2m,sp,tcc
mean,68.43237,0.0075862897,289.01578,98356.58,0.7180500543958317
std,20.80608,0.003200552,8.009596,4579.9053,0.3582878712212896
min,-6.650593,4.6829928e-07,237.64813,72037.44,9.713876067595185e-08
max,210.76653,0.023190046,323.23096,105000.81,1.0
median,72.755615,0.00717825,289.05176,100340.625,0.9608127261480828
mad,4.323242,-0.0004080399,0.035980225,1984.0469,0.24276267175225108


# Plot bar plot of statistics as a func of filters 

In [7]:
fig, axes = plt.subplots(len(VARIABLES), 1, figsize = (TEXT_WIDTH_IN, TEXT_HEIGHT_IN), sharex = True)
""" plot_3grouped_bar_plot """

width = 0.4
w = width/2

for i, ax in enumerate(axes.flatten()):
    var = VARIABLES[i]
    TCC = retrive_variable_from_stores_statistics(variable = var)
    TCC.drop('artefact', axis = 0, inplace = True)
    labels = TCC['mean'].index.values
    x_ticks_labels = TCC.transpose()['all'].index.values
    list_y_values = np.array([ TCC.transpose()[lab].values for lab in labels ])
    y_pos = np.arange(len(list_y_values[0]))
    
    #for i, y_values in enumerate(list_y_values):
    rect = ax.bar(y_pos - 3*width/4, list_y_values[0], width=w, align = 'center', alpha = 0.5, label = '{}'.format(labels[0]))
    #autolabel(rect, ax) # add annotations to plot

    rect2 = ax.bar(y_pos - width/4, list_y_values[1], width=w, align='center', alpha=0.5, label='{}'.format(labels[1]))
    #autolabel(rect2, ax, below = False)  # add annotations to plot

    rect3 = ax.bar(y_pos + width/4, list_y_values[2], width=w, align='center', alpha=0.5, label='{}'.format(labels[2]))
    #autolabel(rect3, ax)  # add annotations to plot

    rect4 = ax.bar(y_pos + 3*width/4, list_y_values[3], width=w, align='center', alpha=0.5, label='{}'.format(labels[3]))
    #autolabel(rect4, ax, below = False)  # add annotations to plot
    ax.set_ylabel('{} [{}]'.format(var, UNITS[var]))
    ax.set_title(LONGNAME[var])
    
plt.xticks(y_pos, x_ticks_labels)
plt.legend()
plt.subplots_adjust(hspace= 0.5, top= 0.95, bottom= 0.05, left= 0.15, right= 0.97)
plt.savefig(os.path.join(path_store_plots, 'bar_plot_global_statistics.pdf'))

In [20]:
#plot_3grouped_bar_plot(list_y_values = list_y_values, 
#                       labels = labels, 
#                       x_ticks_labels = x_ticks_labels, 
#                       title = title, 
#                       ylabel = ylabel, 
#                       filename = None)

In [21]:
def plot_bar_plot(data, title, ylabel, filename = None, formatter = '{:.1f}'):
    """ Helper function used to compute bar plots.

    :param y_values:
    :param title:
    :param xlabel:
    :param filename:
    
    :return:

    """

    y_vals = data.values
    y_pos = np.arange(len(y_vals))
    x_ticks_labels = data.index.values

    fig, ax = plt.subplots(1, 1, figsize = (TEXT_WIDTH_IN, TEXT_HEIGHT_IN) )
    rect = ax.bar(y_pos, y_vals, align = 'center', alpha = 0.5)
    ax.set_ylabel(ylabel)
    autolabel(rect, ax, False, formatter) # add annotations to plot
    plt.xticks(y_pos, x_ticks_labels, rotation = 45)
    plt.title(title)
    plt.subplots_adjust(hspace= 0.5, top= 0.9, bottom= 0.2, left= 0.07, right= 0.97)
    #plt.legend()
    #plt.show()
    #if filename:
    plt.savefig(os.path.join(path_store_plots, 'test_bar_plot.png'))

    return


In [9]:
#plot_bar_plot(data = TCC['mean'], title=LONGNAME['tcc'], ylabel='cloud cover', filename = None, formatter = '{:.1f}')

# Pixel

In [10]:
files = glob.glob(path_stats_results + '/test/*pixel*.nc')

In [11]:
files

['/home/hanna/lagrings/results/stats//test/stats_pixel_q_all.nc',
 '/home/hanna/lagrings/results/stats//test/stats_pixel_r_all.nc',
 '/home/hanna/lagrings/results/stats//test/stats_pixel_sp_all.nc',
 '/home/hanna/lagrings/results/stats//test/stats_pixel_t2m_all.nc',
 '/home/hanna/lagrings/results/stats//test/stats_pixel_tcc_all.nc']

In [33]:
data = xr.open_dataset(files[0])

In [34]:
data

<xarray.Dataset>
Dimensions:    (latitude: 81, longitude: 161)
Coordinates:
  * longitude  (longitude) float64 -15.0 -14.75 -14.5 -14.25 ... 24.5 24.75 25.0
  * latitude   (latitude) float64 50.0 49.75 49.5 49.25 ... 30.5 30.25 30.0
Data variables:
    mean       (latitude, longitude) float32 ...
    std        (latitude, longitude) float32 ...
    min        (latitude, longitude) float32 ...
    max        (latitude, longitude) float32 ...
    median     (latitude, longitude) float32 ...

# Plotting global statistics of one property for all variables.

In [22]:
#from sclouds.helpers import path_python_figures
from sclouds.plot.helpers import color_maps, path_python_figures
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)


In [23]:
n_rows = len(VARIABLES)
n_cols = 1
levels_contourplot = 100

for stat in STATISTICS:
    fig, axes =  plt.subplots(nrows = n_rows, ncols = n_cols, sharex=True, sharey=False)
    fig.set_size_inches(w = TEXT_WIDTH_IN, h = TEXT_HEIGHT_IN - 1)

    for var, ax in zip(VARIABLES, axes):
        #if var != 'tcc':
        #print('Warning this duplicates the RH in plot for tcc')
        files = glob.glob(path_stats_results + '/test/*pixel*{}*.nc'.format(var))
        if len(files) != 1:
            print(files)
        data = xr.open_dataset(files[0])
        vals = data[stat].values
        cntours = ax.contourf(vals, levels=levels_contourplot, cmap=color_maps[var])

        # Removes white lines
        for c in cntours.collections:
            c.set_edgecolor("face")

        fig.colorbar(cntours, ax=ax, label = '{} [{}]'.format(var, UNITS[var]))
        #a = sns.heatmap(vals, ax = ax, cbar = True, cmap = 'viridis', linewidths=0)
        ax.set_title(LONGNAME[var], fontsize = 14)
        ax.set_ylabel('Latitude')

        ax.xaxis.set_major_locator(MultipleLocator(20))
        ax.yaxis.set_major_locator(MultipleLocator(20))
        ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))

        ax.set_yticklabels(labels = np.linspace(30, 50, 5))
        ax.set_xticklabels(labels = np.linspace(-20, 25, 10), rotation = 45)
        
    fig.suptitle('{}'.format(stat), fontsize=14)
    plt.xlabel('Longitude')
    plt.subplots_adjust(wspace = 0.2, hspace = 0.2, top=0.97, bottom=0.1, left = 0.14, right = .95)
    plt.savefig(path_python_figures + 'contourplot_all_variables_{}.pdf'.format(stat))


KeyboardInterrupt: 